<img src="https://huggingface.co/datasets/PleIAs/RAG-Resources/resolve/main/spqrllm.png">

<h1 align="center"><i>A demo with Pleias-Pico</i></h1>

What if your LLM knowledge was stuck in the antiquity?

SPQR LLM is a RAG experiment on most ancient greek and roman literature digitized by the Perseus project. For every question, ancient or modern, the model will strive to build up an answer only relying on sources written before the late antiquity, most of them coming from the 450 BC-200 AD period.

This experiment showcases the new capacities of the Pleias knowledge retrieval models, <a href="https://huggingface.co/PleIAs/Pleias-Nano">Nano</a> and <a href="https://huggingface.co/PleIAs/Pleias-Pico">Pico</a>, including built-in support for source analysis, reference and text grounding.

This notebook uses the smaller model, Pico (350 million parameters). While using it you have to remember the model is **smaller than GPT-2-medium**!

Pleias models are only trained on open data which makes them *historical* model trained on a large amount of sources published before 1990. They are expected to have better familiarity with antic texts than any other LLM of similar size.

## Loading the dataset

As usual on colab, we start by doing multiple installs.

In [ ]:
# Setup
!pip install lancedb pandas langchain langchain-community pypdf tiktoken sentence_transformers tantivy==0.20.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchai

We will start by loading the vector dataset. It is a zipped lancedb project containing the embeddings for 2,700 ancient works translated in English computed by bge-m3. Future versions will include more languages.

In [ ]:
!wget https://huggingface.co/datasets/PleIAs/RAG-Resources/resolve/main/latin_greek_text.lance-20241210T164550Z-001.zip?download=true -O latin_greek_text.lance.zip
!unzip latin_greek_text.lance.zip

!mkdir latin_greek_text_db
!mv latin_greek_text.lance latin_greek_text_db/

--2024-12-11 11:12:20--  https://huggingface.co/datasets/PleIAs/RAG-Resources/resolve/main/latin_greek_text.lance-20241210T164550Z-001.zip?download=true
Resolving huggingface.co (huggingface.co)... 13.35.210.61, 13.35.210.66, 13.35.210.77, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.61|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/04/6c/046c7eb7e3371f3b5417a3650d65b05454ee73f45840dc0d0f971861a3c9b415/6eb7adb68e9874caba4d6dae28e690a700596b5ce7926fdcd539efd54b04a9b1?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27latin_greek_text.lance-20241210T164550Z-001.zip%3B+filename%3D%22latin_greek_text.lance-20241210T164550Z-001.zip%22%3B&response-content-type=application%2Fzip&Expires=1734174740&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDE3NDc0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzA0LzZjLzA0NmM3ZWI3ZTMzNzFmM2I1NDE3YTM2NT

We load the dataset using lancedb.

In [ ]:
import os
import getpass
import pandas as pd

import lancedb

from langchain_community.vectorstores import LanceDB
from langchain.embeddings import HuggingFaceEmbeddings
from lancedb.embeddings import get_registry
from lancedb.pydantic import Vector, LanceModel

from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

db = lancedb.connect("latin_greek_text_db")

lancedb_table = db.open_table("latin_greek_text")

lancedb_table

LanceDBConnection(/content/latin_greek_text_db)

We can now run our first query:

In [ ]:
lancedb_table.search("What is the best season for fruits?").limit(10).to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

,hash_id,identifier,title,author,order_text,text,vector,_distance
0,81543aa4556aec77,tlg0007.tlg112.perseus-eng2_7,Symposiacs,Plutarch,42,For both the summer and the early autumn bear ...,"[-0.008282388, 0.015326452, -0.023731086, -0.0...",0.724141
1,ec7bf192ec9d7580,phi0978.phi001.perseus-eng1_72,The Natural History,Pliny the Elder,1,This is the proper time for gathering fruit; t...,"[0.012949209, 0.03070421, -0.04362494, 0.02404...",0.747091
2,cad93af6e9621953,tlg0007.tlg125.perseus-eng2_0,Plutarch's Natural Questions,Plutarch,7,Or is it because heat fighting against cold ca...,"[-0.018504102, 0.04148123, -0.0421267, -0.0017...",0.792306
3,56386670ef04cf5d,phi0978.phi001.perseus-eng1_61,The Natural History,Pliny the Elder,13,The fruits which fall most readily before they...,"[0.012799004, 0.04470042, -0.03663925, -0.0004...",0.807795
4,225cc3cb2d5649a7,phi0978.phi001.perseus-eng1_61,The Natural History,Pliny the Elder,9,"The terebinth, the maple, and the ash produce ...","[-0.014736435, 0.044617243, -0.026414258, 0.00...",0.812626
5,70580de03691ddd3,phi0978.phi001.perseus-eng1_65,The Natural History,Pliny the Elder,33,"However, there are certain fixed periods of th...","[-0.0076793395, 0.0144542465, -0.030387813, 0....",0.821087
6,42748fad4dc9494c,tlg0008.tlg001.perseus-eng2_4,The Deipnosophists,Athenaeus,64,"And moreover, that of this fruit those which a...","[0.026232745, 0.04473641, -0.03572028, 0.00964...",0.822074
7,a133de7529136c19,phi0845.phi002.perseus-eng1_7,"On Agriculture, Books 1-9",Columella,59,"The carob-tree, which some people called Cerat...","[0.03852992, 0.015690932, -0.0150162205, 0.023...",0.824181
8,c3f6bce355cb9754,phi0978.phi001.perseus-eng1_61,The Natural History,Pliny the Elder,20,"This, however, is constantly to be witnessed i...","[0.019465365, 0.037772123, -0.05108791, -0.012...",0.825922
9,9aeeed73cefc3fa7,phi0978.phi001.perseus-eng1_63,The Natural History,Pliny the Elder,33,Those trees which are the slowest in bringing ...,"[0.020600302, 0.050828453, -0.045996573, 0.001...",0.837583


Lancedb also support hybrid queries which can yield better results on collections of texts using a specialized language (basically you don't really want to substitute one word with the other). For this specific project, it's better overall to stick to embeddings as we rather want to maximize semantic proximity regardless of the words used.

In [ ]:
lancedb_table.search("What is the best season for fruits?", query_type="hybrid").limit(10).to_pandas()

TypeError: LanceTable.search() got multiple values for argument 'query'

## Connecting Pleias-Nano

We load the model from HuggingFace.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tokenizers.decoders import BPEDecoder

model_name = "PleIAs/Pleias-Pico"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/707M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.41k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(65536, 1024)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (k_proj): Linear(in_features=1024, out_features=512, bias=False)
          (v_proj): Linear(in_features=1024, out_features=512, bias=False)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=1024, out_features=2560, bias=False)
          (up_proj): Linear(in_features=1024, out_features=2560, bias=False)
          (down_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((1024,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((1024,), eps=1e-05)
      )
    )
    (norm): 

As you can see it is a llama type model (actually in turn a gpt-neox model since LLama ultimately derives from this family). Yet, it has many specific features that you won't find in other models, including a whole new 65k tokens tokenizer.

We prepare now multiple functions for the RAG generation. Notice that the Pleias RAG model works best with deterministic generation for now on (temperature = 0). Concretely, given a query, you will always get the same answer, unless you tweak it a bit.

In [ ]:
import torch
from typing import List, Optional, Union

class OptimizedGenerator:
    def __init__(self, model, tokenizer, device="cuda"):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device

        # Ensure tokenizer has proper padding settings
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        if self.tokenizer.pad_token_id is None:
            self.tokenizer.pad_token_id = 1

    @torch.inference_mode()
    def generate_single(
        self,
        text: str,
        max_new_tokens: int = 1500,
    ) -> str:
        """Generate text for a single input"""
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            padding=True,
        ).to(self.device)

        outputs = self.model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=max_new_tokens,
            repetition_penalty=1,
            do_sample=False,
            early_stopping=True,
            use_cache=True,
            pad_token_id=self.tokenizer.pad_token_id,
            eos_token_id=2
        )

        return self.tokenizer.decode(outputs[0], skip_special_tokens=False)

    def __call__(
        self,
        texts: Union[str, List[str]],
        batch_size: int = 4,
        max_new_tokens: int = 1500,
    ) -> Union[str, List[str]]:
        """Convenience method to handle both single and batch inputs"""
        if isinstance(texts, str):
            return self.generate_single(texts, max_new_tokens)
        return self.generate_batch(texts, batch_size, max_new_tokens)

def rag_generation(query, max_new_tokens=1500):
    import json
    import torch

    # Ensure pad_token and pad_token_id are set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = 1

    search_results = lancedb_table.search(query).limit(5).to_pandas()

    rag_submission = [f"<|query_start|>{query}<|query_end|>"]

    citation_list = {}
    for index, result in search_results.iterrows():
      citation_list[result["hash_id"]] = result["hash_id"] + " [" + result["author"] + ", " + result["title"] + "]"
      entry = f"<|source_start|><|source_id_start|>" + result["hash_id"] + "<|source_id_end|>" + result["text"] + "<|source_end|>"
      rag_submission.append(entry)

    prompt = "\n".join(rag_submission) + "\n<|source_analysis_start|>"

    result = generator(prompt)

    return result, citation_list

def rag_processing(query):

  rag_result, citation_list = rag_generation(query, generator)

  source, generation = rag_result.split("<|source_analysis_start|>")
  analysis, answer = generation.split("<|answer_start|>")
  analysis, answer = analysis.replace("<|source_analysis_end|>\n", ""), answer.replace("<|answer_end|><|end_of_text|>", "")

  # Regular expression to find references
  ref_pattern = r'<ref name="(.*?)">(.*?)</ref>'

  import re

  # Extract references and replace them with numbered calls
  footnotes = []
  def replace_reference(match):
      hash_value, quoted_text = match.groups()
      footnotes.append(f"[{len(footnotes) + 1}] {citation_list[hash_value]}: {quoted_text}")
      return f"[{len(footnotes)}]"

  # Replace references in the text
  formatted_text = re.sub(ref_pattern, replace_reference, answer)

  # Join footnotes into a single string
  footnotes_text = "<br>".join(footnotes)

  from IPython.display import HTML
  import html

  final_text = "<h2>Analysis</h2><i>" + analysis + "</i><br><br><h2>Answer</h2>" + formatted_text + "<br><br><h2>Bibliography</h2>" + footnotes_text

  # Display as HTML
  return HTML(f'<div style="font-size: 1.1em">{final_text}</div>')

generator = OptimizedGenerator(model, tokenizer)

Before testing, it's always good to assess your query first. Results are order by "cosine distance" to the query (so lower is closer):

In [ ]:
query = "Quae est significatio intelligentiae artificialis?"

results = lancedb_table.search(query).limit(5).to_pandas()

for i, result in results.iterrows():
  print(f"\nResult {i} (Cosine distance: {result['_distance']:.2f}):")
  print(f"Title: {result['title']}")
  print(f"Author: {result['author']}")
  print(f"Text snippet: {result['text'][:300]}...")

NameError: name 'lancedb_table' is not defined

We can now run the text generation. It takes about 30 seconds on a t4 (which is not well optimized yet at all as I'm just using vanilla model generate)

In [ ]:
final_rag_result = rag_processing(query)
display(final_rag_result)

NameError: name 'rag_processing' is not defined